In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from pytorch_forecasting import TimeSeriesDataSet
from pathlib import Path
import matplotlib.pyplot as plt

from helper_functions.model_builder import LSTM
from helper_functions.data_setup import TimeseriesDataset
from helper_functions.engine import train

from torch.utils.data import DataLoader
from torch import nn
from sklearn.metrics import mean_absolute_error as mae

import numpy as np
from numpy import array, split
import pandas as pd
from copy import deepcopy as dc
import warnings
warnings.filterwarnings("ignore")

c:\Users\r0776327\AppData\Local\miniconda3\lib\site-packages\pytorch_forecasting\models\base_model.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
df = pd.read_csv("Datasets/Dataframes_one_ex_var/ModelBenchmark_dataframe.csv")
df.head(), df.shape

(                  Date  Price  FR Generation  FR Load
 0  2018-01-01 00:00:00   4.74        53625.0  56250.0
 1  2018-01-01 01:00:00   3.66        52398.0  54300.0
 2  2018-01-01 02:00:00   1.26        51825.0  53600.0
 3  2018-01-01 03:00:00 -20.10        50729.0  50000.0
 4  2018-01-01 04:00:00 -31.82        50719.0  47100.0,
 (43824, 4))

In [3]:
df = df[["Date", "Price"]]
df.head(2)

,Date,Price
0,2018-01-01 00:00:00,4.74
1,2018-01-01 01:00:00,3.66


In [4]:
df.Date = pd.to_datetime(df.Date)
df.set_index("Date", inplace=True)
df = df.astype("float32")
df.head(2)

,Price
Date,
2018-01-01 00:00:00,4.74
2018-01-01 01:00:00,3.66


In [5]:
# Define the test period
start_test_period = df.index.get_loc("2022-01-01").start
end_test_period = df.index.get_loc("2022-12-31").stop

In [6]:
# split a univariate dataset into train/test sets
def split_dataset(df, start_test_period:int, end_test_period):
	# split into standard weeks
	train, test = df[0:start_test_period], df[start_test_period:end_test_period]
	# restructure into windows of daily data
	train = array(split(train, len(train)/24))
	test = array(split(test, len(test)/24))
	return train, test

# load the new file
train_set, test_set = split_dataset(df.values, start_test_period, end_test_period)
# validate train data
print(train_set.shape) #1461 = 365*3 + 366
print(train_set[0, 0, 0], train_set[-1, -1, 0])
# validate test
print(test_set.shape)
print(test_set[0, 0, 0], test_set[-1, -1, 0])

(1461, 24, 1)
4.74 82.02
(365, 24, 1)
67.07 -4.39


In [7]:
# convert history into inputs and outputs
def to_supervised(train, n_input, n_out=24):
	"""
	train = list of days (history to be considered)
	n_input = number of time steps to use as inputs
	n_out = forecast horizon (here always 24 hour ahead)
	"""
	# flatten data
	data = train.reshape((train.shape[0]*train.shape[1], train.shape[2]))
	X, y = list(), list()
	in_start = 0
	# step over the entire history one time step at a time
	for _ in range(len(data)):
		# define the end of the input sequence
		in_end = in_start + n_input
		out_end = in_end + n_out
		# ensure we have enough data for this instance
		if out_end <= len(data):
			x_input = data[in_start:in_end, 0]
			x_input = x_input.reshape((len(x_input), 1))
			X.append(x_input)
			y.append(data[in_end:out_end, 0])
		# move along one time step
		in_start += 1
	return array(X), array(y)

X_train, y_train = to_supervised(train_set, n_input=24*7)
X_train.shape, y_train.shape

((34873, 168, 1), (34873, 24))

In [8]:
X_test, y_test = to_supervised(test_set, n_input=24*7)
X_test.shape, y_test.shape

((8569, 168, 1), (8569, 24))

In [9]:
import torch
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([34873, 168, 1]),
 torch.Size([34873, 24]),
 torch.Size([8569, 168, 1]),
 torch.Size([8569, 24]))

In [10]:
train_dataset = TimeseriesDataset(X_train, y_train)
test_dataset = TimeseriesDataset(X_test, y_test)

In [11]:
batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

torch.Size([16, 168, 1]) torch.Size([16, 24])


In [13]:
model = LSTM(1, 4, 1)
model.to(device)
model

LSTM(
  (lstm): LSTM(1, 4, batch_first=True)
  (fc): Linear(in_features=4, out_features=1, bias=True)
)

In [ ]:
"""
Contains functions for training and testing a PyTorch model.
"""
import torch

from tqdm.auto import tqdm
from typing import Dict, List, Tuple
from sklearn.metrics import mean_absolute_error as mae

def train_step(model: torch.nn.Module, 
               dataloader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> Tuple[float, float]:
  """Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
    model: A PyTorch model to be trained.
    dataloader: A DataLoader instance for the model to be trained on.
    loss_fn: A PyTorch loss function to minimize.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of training loss and training accuracy metrics.
    In the form (train_loss, train_accuracy). For example:

    (0.1112, 0.8743)
  """
  # Put model in train mode
  model.train(True)

  # Setup train loss and train accuracy values
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # 1. Forward pass
      y_pred = model(X)

      # 2. Calculate  and accumulate loss
      loss = loss_fn(y_pred, y)
      train_loss += loss.item() 

      # 3. Optimizer zero grad
      optimizer.zero_grad()

      # 4. Loss backward
      loss.backward()

      # 5. Optimizer step
      optimizer.step()

      # Calculate and accumulate accuracy metric across all batches
      # avg_loss_accross_batches = train_loss / 100
      train_acc += mae(y_pred.detach(), y)
      # y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
      # train_acc += (y_pred_class == y).sum().item()/len(y_pred)

  # Adjust metrics to get average loss and accuracy per batch 
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader)
  return train_loss, train_acc

def test_step(model: torch.nn.Module, 
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              device: torch.device) -> Tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
    model: A PyTorch model to be tested.
    dataloader: A DataLoader instance for the model to be tested on.
    loss_fn: A PyTorch loss function to calculate loss on the test data.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A tuple of testing loss and testing accuracy metrics.
    In the form (test_loss, test_accuracy). For example:

    (0.0223, 0.8985)
  """
  # Put model in eval mode
  model.eval() 

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
      # Loop through DataLoader batches
      for batch, (X, y) in enumerate(dataloader):
          # Send data to target device
          X, y = X.to(device), y.to(device)

          # 1. Forward pass
          test_pred = model(X)

          # 2. Calculate and accumulate loss
          loss = loss_fn(test_pred, y)
          test_loss += loss.item()          
          
          # Calculate and accumulate accuracy
          test_acc += mae(test_pred.detach(), y)
          
          # test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

  # Adjust metrics to get average loss and accuracy per batch 
  test_loss = test_loss / len(dataloader)
  test_acc = test_acc / len(dataloader)
  return test_loss, test_acc

def train(model: torch.nn.Module, 
          train_dataloader: torch.utils.data.DataLoader, 
          test_dataloader: torch.utils.data.DataLoader, 
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> Dict[str, List]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
    model: A PyTorch model to be trained and tested.
    train_dataloader: A DataLoader instance for the model to be trained on.
    test_dataloader: A DataLoader instance for the model to be tested on.
    optimizer: A PyTorch optimizer to help minimize the loss function.
    loss_fn: A PyTorch loss function to calculate loss on both datasets.
    epochs: An integer indicating how many epochs to train for.
    device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
    A dictionary of training and testing loss as well as training and
    testing accuracy metrics. Each metric has a value in a list for 
    each epoch.
    In the form: {train_loss: [...],
                  train_acc: [...],
                  test_loss: [...],
                  test_acc: [...]} 
    For example if training for epochs=2: 
                 {train_loss: [2.0616, 1.0537],
                  train_acc: [0.3945, 0.3945],
                  test_loss: [1.2641, 1.5706],
                  test_acc: [0.3400, 0.2973]} 
  """
  # Create empty results dictionary
  results = {"train_loss": [],
      "train_acc": [],
      "test_loss": [],
      "test_acc": []
  }

  # Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs)):
      train_loss, train_acc = train_step(model=model,
                                          dataloader=train_dataloader,
                                          loss_fn=loss_fn,
                                          optimizer=optimizer,
                                          device=device)
      test_loss, test_acc = test_step(model=model,
          dataloader=test_dataloader,
          loss_fn=loss_fn,
          device=device)

      # Print out what's happening
      print(
          f"Epoch: {epoch+1} | "
          f"train_loss: {train_loss:.4f} | "
          f"train_MAE: {train_acc:.4f} | "
          f"test_loss: {test_loss:.4f} | "
          f"test_MAE: {test_acc:.4f}"
      )

      # Update results dictionary
      results["train_loss"].append(train_loss)
      results["train_acc"].append(train_acc)
      results["test_loss"].append(test_loss)
      results["test_acc"].append(test_acc)

  # Return the filled results at the end of the epochs
  return results

In [14]:
learning_rate = 0.001
num_epochs = 10
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Start training with help from engine.py
train(model=model,
             train_dataloader=train_loader,
             test_dataloader=test_loader,
             loss_fn=loss_function,
             optimizer=optimizer,
             epochs=num_epochs,
             device=device)

  0%|          | 0/10 [00:00<?, ?it/s]

tensor([[-0.2797],
        [-0.2481],
        [-0.2236],
        [-0.2807],
        [-0.2930],
        [-0.2888],
        [-0.2925],
        [-0.2958],
        [-0.2947],
        [-0.3011],
        [-0.3016],
        [-0.3621],
        [-0.2681],
        [-0.2899],
        [-0.2935],
        [-0.3168]], grad_fn=<AddmmBackward0>) tensor([[ 5.6150e+01,  5.5000e+01,  6.4310e+01,  8.1700e+01,  9.1950e+01,
          1.0004e+02,  1.1270e+02,  9.2900e+01,  9.4480e+01,  8.4680e+01,
          7.9470e+01,  5.5060e+01,  4.3920e+01,  2.2050e+01,  1.5000e+01,
          4.1090e+01,  5.8290e+01,  7.7540e+01,  8.8810e+01,  8.5420e+01,
          7.6640e+01,  6.0610e+01,  6.1380e+01,  5.3880e+01],
        [ 7.8500e+01,  7.0000e+01,  8.4430e+01,  6.7690e+01,  5.3340e+01,
          5.2680e+01,  5.2810e+01,  5.0470e+01,  5.1330e+01,  5.3040e+01,
          5.7260e+01,  5.9800e+01,  6.6920e+01,  6.7710e+01,  6.8000e+01,
          6.7010e+01,  6.2530e+01,  6.0000e+01,  6.0000e+01,  6.2780e+01,
          6.992

ValueError: y_true and y_pred have different number of output (1!=24)